<a href="https://colab.research.google.com/github/kim1987/aiffel/blob/main/aiffel/deeper/deep_two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 14.0 MB/s 


In [37]:
!pip install torchinfo

In [2]:
import os

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np

%matplotlib inline

In [3]:
import tensorflow as tf
from tqdm.notebook import tqdm

In [4]:
import sentencepiece as spm
import pandas as pd

In [38]:
from torchinfo import summary

In [5]:
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

--2021-12-08 08:48:37--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings_train.txt’

ratings_train.txt   100%[===================>]  13.95M  --.-KB/s    in 0.07s   

2021-12-08 08:48:39 (198 MB/s) - ‘ratings_train.txt’ saved [14628807/14628807]

--2021-12-08 08:48:39--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Saving

In [6]:
train_pd = pd.read_table('/content/ratings_train.txt')
test_pd = pd.read_table('/content/ratings_test.txt')

In [7]:
train_pd['document'].to_csv('/content/corpus', sep=' ', index=False,header=False)

        bos_token="<s>",
        eos_token="</s>",
        unk_token="<unk>",
        sep_token="<sep>",
        pad_token="<pad>",
        cls_token="<cls>",
        mask_token="<mask>",
        additional_special_tokens=["<eop>", "<eod>"]

f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" +
    " --pad_id=0 --pad_piece=[PAD]" +
    " --unk_id=1 --unk_piece=[UNK]" +
    " --bos_id=2 --bos_piece=[BOS]" +
    " --eos_id=3 --eos_piece=[EOS]" +
    " --user_defined_symbols=[SEP],[CLS],[MASK]")

In [8]:
spm.SentencePieceTrainer.train(input='/content/corpus', model_prefix='m', vocab_size=10000, model_type='bpe',
                               user_defined_symbols=['<sep>', '<cls>, <mask>'],
                               pad_id=0, pad_piece='<pad>',
                               unk_id=1, unk_piece='<unk>',
                               bos_id=2, bos_piece='<bos>',
                               eos_id=3, eos_piece='<eos>'                               
                               )

In [9]:
s = spm.SentencePieceProcessor()
s.Load('/content/m.model')

True

In [10]:
s.encode_as_ids(train_pd['document'][0])

[12, 1266, 7, 83, 2620, 36, 2188]

In [11]:
train_pd['document'].apply(lambda x: type(x)!=str).sum()

5

In [12]:
train_pd['ids']=train_pd['document'].apply(lambda x : s.encode_as_ids(str(x)))
test_pd['ids']=test_pd['document'].apply(lambda x : s.encode_as_ids(str(x)))

In [13]:
print(train_pd['ids'].apply(len).max())
print(test_pd['ids'].apply(len).max())

123
112


In [14]:
max_len=130

In [15]:
train_pd['ids']=pd.Series(tf.keras.preprocessing.sequence.pad_sequences(train_pd['ids'],
                                                        value=0,
                                                        padding='post',
                                                        maxlen=max_len).tolist())
test_pd['ids']=pd.Series(tf.keras.preprocessing.sequence.pad_sequences(test_pd['ids'],
                                                        value=0,
                                                        padding='post',
                                                        maxlen=max_len).tolist())

데이터셋, 모델

In [16]:
class MyDataset(torch.utils.data.Dataset):
  def __init__(self,data):
    self.ids = data['ids']
    self.label = data['label']
  
  def __len__(self):
    return len(self.ids)

  def __getitem__(self,idx):
    return torch.tensor(self.ids[idx]),torch.tensor(self.label[idx])




In [17]:
class MyModel(nn.Module):
  def __init__(self,vocab_size,max_len):
    super().__init__()
    self.embed = nn.Embedding(num_embeddings = vocab_size,embedding_dim=128,padding_idx=0)
    self.lstm = nn.LSTM(input_size=128,hidden_size=256,num_layers=2,batch_first=True,proj_size=8) #output 130*8
    self.linear1 = nn.Linear(130*8,8)
    self.linear2 = nn.Linear(8,2)

  def forward(self,inputs):
    x = self.embed(inputs)
    x = self.lstm(x)
    x = self.linear1(x[0].flatten(start_dim=1))
    x = nn.ReLU()(x)
    return self.linear2(x)

In [42]:
summary(MyModel(10000,max_len),input_type=torch.long,batch_size=128)

Layer (type:depth-idx)                   Param #
MyModel                                  --
├─Embedding: 1-1                         1,280,000
├─LSTM: 1-2                              163,840
├─Linear: 1-3                            8,328
├─Linear: 1-4                            18
Total params: 1,452,186
Trainable params: 1,452,186
Non-trainable params: 0

훈련준비

In [43]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [44]:
model = MyModel(vocab_size=10000,max_len=max_len).to(device)

In [45]:
train_set = MyDataset(train_pd)
test_set = MyDataset(test_pd)

In [46]:
train_loader = torch.utils.data.DataLoader(train_set,shuffle=True,batch_size=128)
test_loader = torch.utils.data.DataLoader(test_set,shuffle=True,batch_size=128)

In [47]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)

In [52]:
epochs = 3

In [53]:
for epoch in range(epochs):
  model.train()
  total=0
  correct = 0
  for sen, label in tqdm(train_loader):
    optimizer.zero_grad()
    output = model(sen.to(device))
    loss = loss_fn(output,label.to(device))
    loss.backward()
    optimizer.step()
    total+=128
    correct += (output.argmax(1)==label.to(device)).float().sum()
  print('epoch accuracy = ', correct/total)

  0%|          | 0/1172 [00:00<?, ?it/s]

epoch accuracy =  tensor(0.8597, device='cuda:0')


  0%|          | 0/1172 [00:00<?, ?it/s]

epoch accuracy =  tensor(0.8781, device='cuda:0')


  0%|          | 0/1172 [00:00<?, ?it/s]

epoch accuracy =  tensor(0.8914, device='cuda:0')


In [54]:
model.eval()
total=0
correct = 0
with torch.no_grad():
  for sen, label in tqdm(test_loader):
    output = model(sen.to(device))
    total+=128
    correct += (output.argmax(1)==label.to(device)).float().sum()
print('epoch accuracy = ', correct/total)

  0%|          | 0/391 [00:00<?, ?it/s]

epoch accuracy =  tensor(0.8572, device='cuda:0')
